# Training PassGAN

This Jupyter notebook is mainly for debugging, has same functionality to "train.py". However, the parameters are configured small.

## Dependencies

In [1]:
import os, sys
sys.path.append(os.getcwd())

import time
import pickle
import numpy as np
import tensorflow as tf

import utils
import tflib as lib
import tflib.ops.linear
import tflib.ops.conv1d
import tflib.plot
import models

In [3]:
print(tf.test.is_gpu_available())

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

True


2024-03-16 23:55:05.750119: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-16 23:55:05.750176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3050 Ti Laptop GPU major: 8 minor: 6 memoryClockRate(GHz): 1.485
pciBusID: 0000:01:00.0
2024-03-16 23:55:05.750214: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2024-03-16 23:55:05.750220: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2024-03-16 23:55:05.750228: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2024-03-16 23:55:05.750233: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully op

In [4]:
# TensorFlow virsion
print(tf.__version__)

1.15.0


## Parameters

To simplify migration process between Jupyter Notebook and .py format, we created virtual ArgumentParser class.

In [5]:
def filter_non_ascii(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    filtered_content = ''.join(char for char in content if ord(char) < 128)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(filtered_content)

input_file = "data/Scrapy_Passwords.txt"
output_file = "data/Scrapy_Passwords_cleaned.txt"

filter_non_ascii(input_file, output_file)


In [6]:
def max_line_length(file_path):
    max_length = 0

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_length = len(line.strip())
            max_length = max(max_length, line_length)

    return max_length

file_path = "data/Scrapy_Passwords.txt"
max_length = max_line_length(file_path)
print(f"The maximum number of characters in a line is: {max_length}")

The maximum number of characters in a line is: 105


In [7]:
# Class to virtualize ArgumentParser
class VirtualArgparse:
    
    # Path to dataset
    training_data = "data/Scrapy_Passwords_cleaned.txt"
    
    # Name of directory to output
    output_dir = "scrapy"
    
    save_every = 100   #5000
    iters = 1000   #200000
    batch_size = 64
    seq_length = 25
    layer_dim = 128
    critic_iters = 10
    lamb = 10

In [8]:
# Virtualize ArgumentParser instance
args = VirtualArgparse

## Creating Directories

In [9]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

if not os.path.isdir(os.path.join(args.output_dir, 'checkpoints')):
    os.makedirs(os.path.join(args.output_dir, 'checkpoints'))

if not os.path.isdir(os.path.join(args.output_dir, 'samples')):
    os.makedirs(os.path.join(args.output_dir, 'samples'))

## Importing Dataset

In [10]:
print(args.training_data)
print(args.seq_length)

data/Scrapy_Passwords_cleaned.txt
25


In [11]:
lines, charmap, inv_charmap = utils.load_dataset(
    path=args.training_data,
    max_length=args.seq_length)

loaded 56498 lines in dataset


## Creating Dictionary

In [12]:
# Pickle to avoid encoding errors with json
with open(os.path.join(args.output_dir, 'charmap.pickle'), 'wb') as f:
    pickle.dump(charmap, f)

with open(os.path.join(args.output_dir, 'charmap_inv.pickle'), 'wb') as f:
    pickle.dump(inv_charmap, f)
    
print("Number of unique characters in dataset: {}".format(len(charmap)))

Number of unique characters in dataset: 96


## Modeling Generator

In [13]:
# Define the placeholder using tf.keras.Input()
real_inputs_discrete = tf.keras.Input(shape=[args.batch_size, args.seq_length], dtype=tf.int32)

# Convert discrete inputs to one-hot encoding using TensorFlow operation
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))

# Print the shape of the one-hot encoded inputs
print(real_inputs.shape)

(?, 64, 25, 96)


In [14]:
real_inputs_discrete = tf.placeholder(tf.int32, shape=[args.batch_size, args.seq_length])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))

In [15]:
print(real_inputs_discrete)
print(real_inputs)

Tensor("Placeholder:0", shape=(64, 25), dtype=int32)
Tensor("one_hot_1:0", shape=(64, 25, 96), dtype=float32)


In [16]:
fake_inputs = models.Generator(args.batch_size, args.seq_length, args.layer_dim, len(charmap))
fake_inputs_discrete = tf.argmax(fake_inputs, fake_inputs.get_shape().ndims-1)

## Modeling Discriminator

In [17]:
disc_real = models.Discriminator(real_inputs, args.seq_length, args.layer_dim, len(charmap))
disc_fake = models.Discriminator(fake_inputs, args.seq_length, args.layer_dim, len(charmap))

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = -tf.reduce_mean(disc_fake)

In [18]:
# WGAN lipschitz-penalty
alpha = tf.random_uniform(
    shape=[args.batch_size,1,1],
    minval=0.,
    maxval=1.
)

differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
gradients = tf.gradients(models.Discriminator(interpolates, args.seq_length, args.layer_dim, len(charmap)), [interpolates])[0]
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1,2]))
gradient_penalty = tf.reduce_mean((slopes-1.)**2)
disc_cost += args.lamb * gradient_penalty

gen_params = lib.params_with_name('Generator')
disc_params = lib.params_with_name('Discriminator')

gen_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(gen_cost, var_list=gen_params)
disc_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(disc_cost, var_list=disc_params)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
# Dataset iterator
def inf_train_gen():
    while True:
        np.random.shuffle(lines)
        for i in range(0, len(lines)-args.batch_size+1, args.batch_size):
            yield np.array(
                [[charmap[c] for c in l] for l in lines[i:i+args.batch_size]],
                dtype='int32'
            )

In [20]:
# During training we monitor JS divergence between the true & generated ngram
# distributions for n=1,2,3,4. To get an idea of the optimal values, we
# evaluate these statistics on a held-out set first.
true_char_ngram_lms = [utils.NgramLanguageModel(i+1, lines[10*args.batch_size:], tokenize=False) for i in range(4)]
validation_char_ngram_lms = [utils.NgramLanguageModel(i+1, lines[:10*args.batch_size], tokenize=False) for i in range(4)]
for i in range(4):
    print("validation set JSD for n={}: {}".format(i+1, true_char_ngram_lms[i].js_with(validation_char_ngram_lms[i])))
true_char_ngram_lms = [utils.NgramLanguageModel(i+1, lines, tokenize=False) for i in range(4)]

validation set JSD for n=1: 0.0012527983135280426
validation set JSD for n=2: 0.010837599210729803
validation set JSD for n=3: 0.054586084631164565
validation set JSD for n=4: 0.1342479767039421


## TensorFlow Session

In [21]:
with tf.Session() as session:

    # Time stamp
    localtime = time.asctime( time.localtime(time.time()) )
    print("Starting TensorFlow session...")
    print("Local current time :", localtime)
    
    # Start TensorFlow session...
    session.run(tf.global_variables_initializer())

    def generate_samples():
        samples = session.run(fake_inputs)
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded = []
            for j in range(len(samples[i])):
                decoded.append(inv_charmap[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples

    gen = inf_train_gen()

    for iteration in range(args.iters + 1):
        start_time = time.time()

        # Train Generator
        if iteration > 0:
            _ = session.run(gen_train_op)

        # Train Discriminator
        for i in range(args.critic_iters):
            _data = next(gen)
            _disc_cost, _ = session.run(
                [disc_cost, disc_train_op],
                feed_dict={real_inputs_discrete:_data}
            )

        lib.plot.output_dir = args.output_dir
        lib.plot.plot('time', time.time() - start_time)
        lib.plot.plot('train disc cost', _disc_cost)

        # Output to text file after every 100 samples
        if iteration % 100 == 0 and iteration > 0:

            samples = []
            for i in range(10):
                samples.extend(generate_samples())

            for i in range(4):
                lm = utils.NgramLanguageModel(i+1, samples, tokenize=False)
                lib.plot.plot('js{}'.format(i+1), lm.js_with(true_char_ngram_lms[i]))

            with open(os.path.join(args.output_dir, 'samples', 'samples_{}.txt').format(iteration), 'w') as f:
                for s in samples:
                    s = "".join(s)
                    f.write(s + "\n")

        if iteration % args.save_every == 0 and iteration > 0:
            model_saver = tf.train.Saver()
            model_saver.save(session, os.path.join(args.output_dir, 'checkpoints', 'checkpoint_{}.ckpt').format(iteration))
            print("{} / {} ({}%)".format(iteration, args.iters, iteration/args.iters*100.0 ))

        if iteration == args.iters:
            print("...Training done.")
        
        #if iteration % 100 == 0:
            #lib.plot.flush()

        #lib.plot.tick()
        
# Time stamp
localtime = time.asctime( time.localtime(time.time()) )
print("Ending TensorFlow session.")
print("Local current time :", localtime)


Starting TensorFlow session...
Local current time : Sat Mar 16 23:57:09 2024



2024-03-16 23:57:09.760380: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-16 23:57:09.760461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3050 Ti Laptop GPU major: 8 minor: 6 memoryClockRate(GHz): 1.485
pciBusID: 0000:01:00.0
2024-03-16 23:57:09.760585: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2024-03-16 23:57:09.760594: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2024-03-16 23:57:09.760602: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2024-03-16 23:57:09.760607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully op

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Discriminator.Input/conv1d (defined at home/jarvis/miniconda3/envs/PassGAN/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[add_21/_3]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Discriminator.Input/conv1d (defined at home/jarvis/miniconda3/envs/PassGAN/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Discriminator.Input/conv1d':
  File "home/jarvis/miniconda3/envs/PassGAN/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "home/jarvis/miniconda3/envs/PassGAN/lib/